# Pipeline avancé

## Intro

* Etre capable de traiter des dataset hétérogènes avec des variables continues, discretes, des mots...

Trois fonctions
1. make_column_transformer
1. make_colums_selector
1. make_union



In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
import seaborn as sns

In [19]:
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [20]:
y = titanic["survived"]
X = titanic.drop("survived", axis=1)

In [21]:
# Ce code part en erreur
# model = make_pipeline(StandardScaler(), SGDClassifier())
# model.fit(X, y)

Y a différents type de variables
* Continue : age
* Discrete : pclass
* Des mots : sex

Si on veut traiter le dataset avec un Transformer, va falloir trier les colonnes
Par exemple StandScaler veut des variables numériques donc on peut pas lui passer "Sex"

Quand on construit un Pipeline avec au bout un estimateur on veut faire passer le dataset en entier afin de pouvoir utiliser toutes les colonnes et entrainer l'Estimator.

En l'état le StandardScaler va retourner une erreur

Il faut donc créer dans le pipeline un mécanisme qui permet de trier les colonnes afin de passer certaines colonnes à certains transformers




## make_column_transformer
* Issue du module compose de sklearn
* Permet d'appliquer des transformations que sur certaines colonnes

In [22]:
from sklearn.compose import make_column_transformer

# Dans un tuple
# Le transformer puis la liste des colonnes à traiter
transformer = make_column_transformer((StandardScaler(), ["age", "fare"]))

# On peut allors utiliser .fit() ou .fit_transform() sur le Transformer qu'on vient de créer
# On lui passe alors tout X mais il ne
transformer.fit_transform(X)





array([[-0.53037664, -0.50244517],
       [ 0.57183099,  0.78684529],
       [-0.25482473, -0.48885426],
       ...,
       [        nan, -0.17626324],
       [-0.25482473, -0.04438104],
       [ 0.15850313, -0.49237783]])

On pourrait écrire vouloir écrire

In [ ]:
#model = make_pipeline(transformer, SGDClassifier())
#model.fit(X, y)

* Mais on va plutôt créer d'autres transformers.
* On va trier les columns qui sont numériques et celles qui sont des catégories
* Créer des pipelines catégoriels avec make_pipeline 
* Les appliquer avec make_column_transformer()

In [28]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

titanic = sns.load_dataset("titanic")
titanic.head()

y = titanic["survived"]
X = titanic.drop("survived", axis=1)

numerical_features = ["pclass", "age", "fare"]

# SimpleImputer pour enlever les valeur manquantes
numerical_pipeline = make_pipeline (SimpleImputer(),
                                    StandardScaler() 
                                   )
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                      )

# On crée le pieline final
model = make_pipeline (preprocessor, SGDClassifier())
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare'])])),
                ('sgdclassifier', SGDClassifier())])

Il faut garder en tête que pour le modèle on utilise QUE les données numériques.

Ci-dessous on crée 2 pipelines : données numériques, données catégorielles

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

titanic = sns.load_dataset("titanic")
titanic.head()

y = titanic["survived"]
X = titanic.drop("survived", axis=1)

numerical_features = ["pclass", "age", "fare"]
categorical_features = ["sex", "deck", "alone"]

# SimpleImputer pour enlever les valeur manquantes
numerical_pipeline = make_pipeline (SimpleImputer(),
                                    StandardScaler() 
                                   )

categorical_pipeline = make_pipeline (SimpleImputer(strategy = "most_frequent"),
                                      OneHotEncoder() 
                                     )


preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                      (categorical_pipeline, categorical_features)
                                      )

# On crée le pieline final
model = make_pipeline (preprocessor, SGDClassifier())
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])

### Résumé
1. Faire le tri entre les valeurs numériques et catégorielles
1. Créer 2 sous-pipeline, 2 chaines de transformation pour chacune des catégories - make_pipeline() - On liste les Transformers spécifiques à appliquer
1. Créer un préprocesseur pour traiter les colonnes - make_column_transformer
1. Créer un pipeline qui contient le preprocessor et le classifier
1. Entrainer le modèle - model.fit

## make_column_selector

* Depuis ver 0.22
* Evite de devoir lister explicitement les noms des colonnes quand on fait le tri entre numerical_features et categorical_features par exemple.
* utile quand on a 100 variables/colonnes

In [26]:
from sklearn.compose import make_column_selector

titanic = sns.load_dataset("titanic")
titanic.head()

y = titanic["survived"]
X = titanic.drop("survived", axis=1)

# numerical_features = ["pclass", "age", "fare"]
# on peut même passer un pattern de type expression régulière pour selectionner les
# colonne commence par Zoubida_xxxx
numerical_features = make_column_selector(dtype_include=np.number)

# categorical_features = ["sex", "deck", "alone"]
categorical_features = make_column_selector(dtype_exclude=np.number)

# SimpleImputer pour enlever les valeur manquantes
numerical_pipeline = make_pipeline (SimpleImputer(),
                                    StandardScaler() 
                                   )

# OneHotEncoder evite chat > chien > oiseau
categorical_pipeline = make_pipeline (SimpleImputer(strategy = "most_frequent"),
                                      OneHotEncoder() 
                                     )


preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                      (categorical_pipeline, categorical_features)
                                      )

### Important
Du coup le modèle est entrainé sur TOUTES les colonnes numériques et sur TOUTES les colonnes catégorielles et pas uniquement sur des sous-ensemble

## make_union - Pipelines parallèles

In [37]:
from sklearn.pipeline import make_union
from sklearn.preprocessing import Binarizer

titanic = sns.load_dataset("titanic")
print(titanic.shape)

y = titanic["survived"]
X = titanic.drop("survived", axis=1)

numerical_features = X[["age", "fare"]]

# On fait passer les données par SimpleImputer puis en \\ on applique StandardScaler() et Binarizer()
# Binarizer() convertit en 0 ou 1 selon un treshold (0 par défaut)
pipeline = make_union(StandardScaler(), Binarizer())
MyTransformer = make_pipeline (SimpleImputer(), pipeline ) 

# Bien voir qu'à la fin il n'y a plus que 4 colonnes
print(MyTransformer.fit_transform(numerical_features).shape)

# À gauche age et fare standardisés, à droite age et fare binarisés
print(MyTransformer.fit_transform(numerical_features))

(891, 15)
(891, 4)
[[-0.5924806  -0.50244517  1.          1.        ]
 [ 0.63878901  0.78684529  1.          1.        ]
 [-0.2846632  -0.48885426  1.          1.        ]
 ...
 [ 0.         -0.17626324  1.          1.        ]
 [-0.2846632  -0.04438104  1.          1.        ]
 [ 0.17706291 -0.49237783  1.          1.        ]]
